In [27]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin

geom = Odin.createGeometry( 'rectangle', [2,2] )
mesh = Odin.createMesh( geom, res=1 )



movingWallVelocity = 2

# make just a list, give a name as optional parameter to flow model
# make this a flowmodeles constructor with kwargs here and then remove simulation instance
myFlowModels = {
    'u' : Odin.TransportModels.staggeredTransport_u,
    'v' : Odin.TransportModels.staggeredTransport_v,
    'p' : Odin.PressureModels.Pressure
}

passiveFields = {}

sim = Odin.Simulation(flowmodels=myFlowModels, mesh=mesh, geometry=geom, passiveFields=passiveFields )


myFields = sim.fieldRegistry
u=myFields['u']
v=myFields['v']
p=myFields['p']

# relates to viscosity:
u.govModel.setDiffusionCoefficient(1)
v.govModel.setDiffusionCoefficient(1)

u.data.fill(0.0)
Odin.defineBoundaryCondition(field=u, fieldReg=myFields, boundaryName='top', boundaryType='fixedValue', value=movingWallVelocity)
Odin.defineBoundaryCondition(field=u, fieldReg=myFields, boundaryName='bottom', boundaryType='fixedValue', value=0 )
Odin.defineBoundaryCondition(field=u, fieldReg=myFields, boundaryName='right', boundaryType='fixedValue', value=0 )
Odin.defineBoundaryCondition(field=u, fieldReg=myFields, boundaryName='left', boundaryType='fixedValue', value=0)

v.data.fill(0.0)
Odin.defineBoundaryCondition(field=v, fieldReg=myFields, boundaryName='top', boundaryType='fixedValue', value=0)
Odin.defineBoundaryCondition(field=v, fieldReg=myFields, boundaryName='bottom', boundaryType='fixedValue', value=0)
Odin.defineBoundaryCondition(field=v, fieldReg=myFields, boundaryName='left', boundaryType='fixedValue', value=0 )
Odin.defineBoundaryCondition(field=v, fieldReg=myFields, boundaryName='right', boundaryType='fixedValue', value=0)

for i in range(1):
    u.data = Odin.solve(u)
    v.data = Odin.solve(v)
    p.data = Odin.solve(p)

sim.display(u,mesh)
sim.display(v,mesh)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 

In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)